In [1]:
import json
from os import listdir
from os.path import isfile, join
import re
import string
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# import openai
import time
# from sentence_transformers import SentenceTransformer
# import tiktoken
# from openai.embeddings_utils import get_embedding
from tqdm import tqdm


[nltk_data] Downloading package punkt to /Users/admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/admin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
path = "../iclr_2017/train/parsed_pdfs" 
train_files = [join(path,f) for f in listdir(path) if isfile(join(path, f))]

path_2 = "../iclr_2017/train/reviews" 
train_reviews = [join(path_2,f) for f in listdir(path_2) if isfile(join(path_2, f))]

path = "../iclr_2017/dev/parsed_pdfs" 
val_files = [join(path,f) for f in listdir(path) if isfile(join(path, f))]

path_2 = "../iclr_2017/dev/reviews" 
val_reviews = [join(path_2,f) for f in listdir(path_2) if isfile(join(path_2, f))]

path = "../iclr_2017/test/parsed_pdfs" 
test_files = [join(path,f) for f in listdir(path) if isfile(join(path, f))]

path_2 = "../iclr_2017/test/reviews" 
test_reviews = [join(path_2,f) for f in listdir(path_2) if isfile(join(path_2, f))]


In [3]:
train_files.sort()
train_reviews.sort()
val_files.sort()
val_reviews.sort()
test_files.sort()
test_reviews.sort()


train_file_list = list(zip(train_files,train_reviews))
val_file_list = list(zip(val_files,val_reviews))
test_file_list = list(zip(test_files,test_reviews))


In [4]:
train_dataset = []
i=0
for file in tqdm(train_file_list):
    i+=1
    t = file[0].split('/')[-1]
    f = open(file[0])
    data = json.load(f)
    title = data['metadata']['title']
    textdata=""
    for section in data['metadata']['sections']:
        if section["heading"]!=None:
            temp = re.sub(r"\n[0-9][0-9][0-9]"," ",section["text"])
            temp = re.sub(r"\n[0-9]"," ",temp)
            temp = re.sub(r'(?<=\w)-\n|\n-(?=\w)', '',temp,flags=re.IGNORECASE) 
            temp = re.sub(r'et al.'," ",temp,flags=re.IGNORECASE)
            temp = re.sub("/n",".",temp)
            #temp = re.sub(r"[!@#$%^&*∑]/g","",temp)
            textdata = textdata+temp
    label = open(file[1])
    lab = json.load(label)
    flag = 1 if lab['accepted'] else 0
    train_dataset.append([t,title,textdata.lower(),flag])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 349/349 [00:00<00:00, 427.08it/s]


In [5]:
len(train_dataset)

349

In [6]:
val_dataset = []
i=0
for file in tqdm(val_file_list):
    i+=1
    t = file[0].split('/')[-1]
    f = open(file[0])
    data = json.load(f)
    textdata=""
    title = data['metadata']['title']
    if data['metadata']['sections'] == None:
        continue
    for section in data['metadata']['sections']:
        if section["heading"]!=None:
            temp = re.sub(r"\n[0-9][0-9][0-9]"," ",section["text"])
            temp = re.sub(r"\n[0-9]"," ",temp)
            temp = re.sub(r'(?<=\w)-\n|\n-(?=\w)', '',temp,flags=re.IGNORECASE) 
            temp = re.sub(r'et al.',"",temp,flags=re.IGNORECASE)
            temp = re.sub("/n",".",temp)
            #temp = re.sub(r"[!@#$%^&*∑]/g","",temp)
            textdata = textdata+temp
    label = open(file[1])
    lab = json.load(label)
    flag = 1 if lab['accepted'] else 0
    val_dataset.append([t,title,textdata.lower(),flag])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 309.25it/s]


In [7]:
test_dataset = []
i=0
for file in tqdm(test_file_list):
    i+=1
    t = file[0].split('/')[-1]
    f = open(file[0])
    data = json.load(f)
    textdata=""
    title = data['metadata']['title']
    for section in data['metadata']['sections']:
        if section["heading"]!=None:
            temp = re.sub(r"\n[0-9][0-9][0-9]"," ",section["text"])
            temp = re.sub(r"\n[0-9]"," ",temp)
            temp = re.sub(r'(?<=\w)-\n|\n-(?=\w)', '',temp,flags=re.IGNORECASE) 
            temp = re.sub("/n",".",temp)
            temp = re.sub(r'et al.',"",temp,flags=re.IGNORECASE)
            #temp = re.sub(r"[!@#$%^&*∑]/g","",temp)
            textdata = textdata+temp
    label = open(file[1])
    lab = json.load(label)
    flag = 1 if lab['accepted'] else 0
    test_dataset.append([t,title,textdata.lower(),flag])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 317.56it/s]


In [8]:
train_dataframe = pd.DataFrame(train_dataset)
val_dataframe = pd.DataFrame(val_dataset)
test_dataframe = pd.DataFrame(test_dataset)


train_dataframe.columns = ['File_id','Title','Paper_text', 'Status']
val_dataframe.columns = ['File_id','Title','Paper_text', 'Status']
test_dataframe.columns = ['File_id','Title','Paper_text', 'Status']

In [9]:
train_dataframe.to_csv("../complete_sentence/train_processed_data.csv",index=False)
val_dataframe.to_csv("../complete_sentence/val_processed_data.csv",index=False)
test_dataframe.to_csv("../complete_sentence/test_processed_data.csv",index=False)

In [10]:
train_dataframe.Status.value_counts()

0    210
1    139
Name: Status, dtype: int64

In [11]:
print(len(train_dataframe['Paper_text'][0].split(".")))
for i in train_dataframe['Paper_text'][0].split("."):
    try:
        len(i)
    except:
        print(i)

397


In [12]:
li=train_dataframe['Paper_text'][0].split(".")

In [13]:
print(len(li))

397


In [14]:
def sentence_split(dataframe):
    train_lis=[] #[id, sentence , label]

    for index,row in tqdm(dataframe.iterrows()):
        li=train_dataframe['Paper_text'][0].split(".")
        for sentence in li:
                if len(sentence)==0:
                    continue
                train_lis.append([row['File_id'],sentence,row['Status']])

    dataframe_list = pd.DataFrame(train_lis)
    dataframe_list.columns = ['File_id','Sentence', 'Status']
    return dataframe_list

In [15]:
train_dataframe_li= sentence_split(train_dataframe)
val_dataframe_li= sentence_split(val_dataframe)
test_dataframe_li= sentence_split(test_dataframe)

349it [00:00, 518.76it/s]
39it [00:00, 627.19it/s]
38it [00:00, 629.57it/s]


In [16]:
train_dataframe_li.to_csv('../sentence_split/train_sentence_data.csv',index=False)
val_dataframe_li.to_csv('../sentence_split/val_sentence_data.csv',index=False)
test_dataframe_li.to_csv('../sentence_split/test_sentence_data.csv',index=False)

In [17]:
val_dataframe_li.Status.value_counts()

0    8316
1    7128
Name: Status, dtype: int64